In [1]:
import pandas3js as pjs
print(pjs.__version__)
import jsonextended as ejson
print(ejson.__version__)
import pandas as pd
import numpy as np
import pythreejs as tjs

0.0.3
0.1.3.3


In [2]:
data = ejson.json_to_dict(ejson.get_test_path())
ejson.dict_pprint(data, depth=3)

dir1: 
  dir1_1: 
    file1_1: {...}
  file1: 
    initial: {...}
    meta: {...}
    optimised: {...}
    units: {...}
  file2: 
    initial: {...}
    meta: {...}
    optimisation: {...}
    optimised: {...}
    units: {...}
dir2: 
  file1: 
    initial: {...}
    meta: {...}
    optimisation: {...}
    optimised: {...}
    units: {...}
dir3: 


In [3]:
data = ejson.dict_remove_paths(data, ['units'])
energies = ejson.dict_filter_keys(data, ['energy'])
pd.Series(ejson.dict_flatten(energies)).describe()

count       89.000000
mean    -24062.286741
std          0.012361
min     -24062.293964
25%     -24062.293727
50%     -24062.293491
75%     -24062.278072
max     -24062.207939
dtype: float64

In [4]:
energies = ejson.units.apply_unitschema(energies, {'energy':'eV'})
energies = ejson.units.apply_unitschema(energies, {'energy':'kcal'},as_quantity=False)
pd.Series(ejson.dict_flatten(energies)).describe()

count    8.900000e+01
mean    -9.214157e-19
std      4.733481e-25
min     -9.214159e-19
25%     -9.214159e-19
50%     -9.214159e-19
75%     -9.214153e-19
max     -9.214126e-19
dtype: float64

In [5]:
optimisation = ejson.dict_multiindex(data,['dir1','file2','optimisation','steps'])
optsteps = sorted(optimisation.keys(), key=ejson.core._natural_keys)
print(optsteps)

['2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '90']


In [10]:
ejson.dict_pprint(optimisation['2'],depth=None,no_values=True)

crystallographic: 
  geometry: 
    assym:         
    atomic_number: 
    id:            
    label:         
    x/a:           
    y/b:           
    z/c:           
  lattice_parameters: 
    a:     
    alpha: 
    b:     
    beta:  
    c:     
    gamma: 
  volume: 
energy: 
primitive: 
  density: 
  geometry: {...}
  lattice_parameters: {...}
  volume:  


In [7]:
ejson.dict_to_html(optimisation['2'])

In [2]:
import ipywidgets as widgets

#TODO dynamic view size

def create_config_gui(data, change_func, 
                      add_objects=True, add_labels=True,
                      view=(10,-10,-10,10),near=-10):
    """ a simple gui to handle configuration changes
    """
    controls = []
    
    # sort number strings correctly
    dkeys = sorted(data.keys(), key=pjs.utils.natural_keys)

    gcollect = pjs.GeometricCollection()
    scene = pjs.create_js_scene_view(gcollect,
                    add_objects=add_objects,add_labels=add_labels)
    camera, renderer = pjs.create_jsrenderer(scene,view=view,near=near)
    select = widgets.SelectionSlider(description='Configuration:',
                options=dkeys, continuous_update=False)
    def handle_slider(change):
        with renderer.hold_trait_notifications():
            geometry = change_func(data[change.new])
            gcollect.change_by_df(geometry,otype_column='otype',
                        otype_default='pandas3js.Sphere')        
    select.observe(handle_slider,names='value')
    select.value = dkeys[-1]
    controls.append(select)
    
    if add_labels:    
        toggle=widgets.Checkbox(
        value=False,
        description='View Label:')
        def handle_toggle(change):
            for obj in gcollect.idobjects:
                obj.label_visible = change.new
        toggle.observe(handle_toggle,names='value')
        controls.append(toggle)
    
    return widgets.VBox([widgets.HBox(controls),renderer])


In [5]:
1/np.array([1,2,3])

array([ 1.        ,  0.5       ,  0.33333333])

In [9]:
from scipy.spatial import cKDTree
cKDTree.query_pairs

def format_label(s):
    s = ''.join([i for i in s if not i.isdigit()])
    return s.lower().capitalize()

def change_crystal_opt(data):
    
    geotype = 'primitive'#'crystallographic'

    ldict = data[geotype]['lattice_parameters']
    
    gdict = data[geotype]['geometry']
    df = pd.DataFrame(gdict)
    
    df['x'] = df['x/a']*ldict['a']
    df['y'] = df['y/b']*ldict['b']
    df['z'] = df['z/c']*ldict['c']
    df.label = df.label.apply(format_label)
    
    atom_df = pjs.atom.atomic_data()
    df['color'] = df['atomic_number'].map(lambda n: atom_df.loc[n].color)
    df['label_color'] = df['color']
    df['radius'] = df['atomic_number'].map(lambda n: atom_df.loc[n].RCov)
    df['transparency'] = 1
    
    df = df[['id','x','y','z',
             'label','label_color',
             'color','radius']]
    df['transparency'] = 1
    df['otype'] = 'pandas3js.Sphere'
    
    # s coordination
    r_array = df[df.label=='S'][['x','y','z']].values    
    ck = cKDTree(r_array)
    pairs = ck.query_pairs(4)
    for i,j in pairs:
        i,j
        series = pd.Series({
            'start':tuple(r_array[i]),'end':tuple(r_array[j]),
            'color':atom_df.loc[16].color,'otype':'pandas3js.Line',
            'id':df['id'].max()+1,'linewidth':3})
        df = df.append(series,ignore_index=True,)

    # lattice vectors
    a_vec, b_vec, c_vec = pjs.atom.vectors_from_params(
        *[ldict[s] for s in ('a','b','c','alpha','beta','gamma')])
    
    series = pd.Series({'a':tuple(a_vec),'b':tuple(b_vec),'c':tuple(c_vec),
                       'otype':'pandas3js.WireBox','id':df['id'].max()+1,'color':'black'})
    
    df = df.append(series,ignore_index=True,)

    return df

In [10]:
data = ejson.json_to_dict(ejson.get_test_path(),['dir1','file2','optimisation','steps'])
create_config_gui(data, change_crystal_opt,add_labels=True,view=(15, -15, -15, 15),near=-15)

In [14]:
df = pd.DataFrame({'a':[1,2], 'b':[3,4]})
df
s = pd.Series({'c':5})
df = df.append(s,ignore_index=True,)
df

,a,b,c
0,1.0,3.0,NaN
1,2.0,4.0,NaN
2,NaN,NaN,5.0


In [ ]:
debug

> /Users/cjs14/GitHub/pandas3js/pandas3js/views/jsmesh.py(131)trait_dlink()
    129     def trait_dlink(change):
    130         func = str_to_obj(dic['func'])
--> 131         value = func(*[getattr(gobject,v) for v in dic['vars']])
    132         jsobject.set_trait(key, value)
    133     # initialise jsobject to correct trait value

ipdb> dic
{'vars': ('x', 'y', 'z', 'a', 'b', 'c'), 'func': 'pandas3js.views.jsmesh._make_box_vertices'}
ipdb> exit


In [8]:
import traitlets as trait
import numpy as np

from pandas3js.utils import str_to_obj, obj_to_str

# rep=representation, var=static variable, 
# dmap=direct-mapping, fmap=functional-mapping
gobject_jsmapping = {
'pandas3js.models.idobject.Sphere':
    {'grep':'pythreejs.SphereGeometry',
     'gvar':{},
     'gdmap':{'radius':'radius'}, 
     'gfmap':{},
     
     'matrep':'pythreejs.LambertMaterial', 
     'matvar':{},
     'matdmap':{'visible':'visible','opacity':'transparency'},
     'matfmap':{'transparent':{'vars':('transparency',),
                               'func':'pandas3js.views.jsmesh._transparent'},
                'color':{'vars':('color',),'func':'matplotlib.colors.to_hex'}},

     'meshrep':'pythreejs.Mesh',
     'meshvar':{},
     'meshdmap':{},
     'meshfmap':{'position':{'vars':('x','y','z'),
                             'func':'pandas3js.views.jsmesh._tolist'}},
    },
'pandas3js.models.idobject.WireBox':
    {'grep':'pythreejs.PlainGeometry',
     'gvar':{},
     'gdmap':{}, 
     'gfmap':{'vertices':{'vars':('x','y','z','a','b','c'),
                          'func':'pandas3js.views.jsmesh._make_box_vertices'},
              'colors':{'vars':('color',),
                        'func':'pandas3js.views.jsmesh._make_box_colors'}},
     
     'matrep':'pythreejs.LineBasicMaterial', 
     'matvar':{'vertexColors':'VertexColors'},
     'matdmap':{'visible':'visible','opacity':'transparency','linewidth':'linewidth'},
     'matfmap':{'transparent':{'vars':('transparency',),
                               'func':'pandas3js.views.jsmesh._transparent'}},

     'meshrep':'pythreejs.Line',
     'meshvar':{'type':'LinePieces'},
     'meshdmap':{},
     'meshfmap':{},
    },
}    

def _create_trait_dlink(dic, key, gobject, jsobject):
    # sync for jsobject to gobject trait value
    def trait_dlink(change):
        func = str_to_obj(dic['func'])
        value = func(*[getattr(gobject,v) for v in dic['vars']])
        jsobject.set_trait(key, value)
    # initialise jsobject to correct trait value
    trait_dlink(None)
    return trait_dlink

def create_jsmesh_view(gobject,mapping=None):
    """create PyThreeJS Text Mesh for GeometricObject
    and with one-way synchronisation
    
    Properties
    ----------
    gobject : GeometricObject
    mapping : None or dict
        if None, use default mapping
    
    Examples
    --------
    
    >>> from pandas3js import Sphere
    >>> sphere = Sphere()
    >>> mesh = create_jsmesh_view(sphere)
    >>> mesh.position
    [0.0, 0.0, 0.0]
    >>> str(mesh.material.color)
    '#ff0000'
    
    >>> sphere.x = 1.0
    >>> mesh.position
    [1.0, 0.0, 0.0]
    
    >>> sphere.color = (1,1,1)
    >>> str(mesh.material.color)
    '#ffffff'
    
    """
    class_str = obj_to_str(gobject)
    if not class_str in gobject_jsmapping:
        raise ValueError('No mapping available for {}'.format(class_str))
    class_map = gobject_jsmapping[class_str]
    
    # create geometry
    geometry = str_to_obj(class_map['grep'])()
    
    for key, val in class_map['gvar'].items():
        geometry.set_trait(key, val)
    for key, val in class_map['gdmap'].items():
        trait.dlink((gobject, val), (geometry, key))
    for gkey, gdic in class_map['gfmap'].items():
        handle = _create_trait_dlink(gdic, gkey, gobject, geometry)
        gobject.observe(handle,names=gdic['vars'])
     
    # create material
    material = str_to_obj(class_map['matrep'])()
    
    for key, val in class_map['matvar'].items():
        material.set_trait(key, val)
    for key, val in class_map['matdmap'].items():
        trait.dlink((gobject, val), (material, key))
    for mkey, mdic in class_map['matfmap'].items():
        handle = _create_trait_dlink(mdic, mkey, gobject, material)
        gobject.observe(handle,names=mdic['vars'])

    # create mesh
    mesh = str_to_obj(class_map['meshrep'])(
                geometry=geometry,material=material)
    
    for key, val in class_map['meshvar'].items():
        mesh.set_trait(key, val)
    for key, val in class_map['meshdmap'].items():
        trait.dlink((gobject, val), (mesh, key))
    for skey, sdic in class_map['meshfmap'].items():
        handle = _create_trait_dlink(sdic, skey, gobject, mesh)
        gobject.observe(handle,names=sdic['vars'])

    return mesh
    

from pandas3js import WireBox
sphere = WireBox()
mesh = create_jsmesh_view(sphere)
print(mesh)
sphere.color = 'pink'
mesh.material.linewidth

1.0

In [4]:
    >>> from pandas3js import Sphere
    >>> sphere = Sphere()
    >>> mesh = create_jsmesh_view(sphere)
    >>> mesh.position
    [0.0, 0.0, 0.0]
    >>> print(mesh.material.color)
    '#ff0000'
    
    #>>> sphere.x = 1.0
    >>> mesh.position
    [1.0, 0.0, 0.0]
    
    >>> sphere.color = 'red'
    >>> print(mesh.material.color)
    '#ffffff'


#ff0000
#ff0000


'#ffffff'

In [103]:
sphere.notify_change()

'white'

TypeError: list() takes at most 1 argument (3 given)

In [20]:
sphere.x = 1.0
mesh.position

(1.0, 0.0, 0.0)


[1.0, 0.0, 0.0]

In [22]:
import numpy as np
o = np.array([0,0,0])
a = np.array([4,0,0])
b = np.array([0,4,0])
c = np.array([0,0,4])

vertices = [o,o+a,
           o,o+b,
           o,o+c,
           o+b, o+b+a,
           o+b, o+b+c,
           o+a, o+b+a,
           o+a, o+a+c,
           o+c, o+c+a,
           o+c, o+c+b,
           o+b+c+a, o+b+c,
           o+b+c+a, o+b+a,
           o+b+c+a, o+c+a,
           ]
geometry = tjs.PlainGeometry(vertices=[v.tolist() for v in vertices],
                             colors = ['red']*len(vertices))
material=tjs.LineBasicMaterial(linewidth=5, vertexColors='VertexColors')
mesh = tjs.Line(geometry=geometry, 
                 material=material, 
                 type='LinePieces')


scene = tjs.Scene(children=[lines, tjs.DirectionalLight(color='#ccaabb', 
                            position=[0,10,0]),tjs.AmbientLight(color='#cccccc')])
c = tjs.PerspectiveCamera(position=[0, 10, 10])
renderer = tjs.Renderer(camera=c, background='gray', background_opacity=1, 
                        scene = scene, controls=[tjs.OrbitControls(controlling=c)])
display(renderer)

In [61]:
debug

> <ipython-input-60-3cf9cf68fc68>(85)create_jsmesh_view()
     83         def mtrait_link(change):
     84             material.set_trait(key, str_to_obj(mdic['func'])(*[getattr(gobject,v) for v in mdic['matvars']]))
---> 85         gobject.observe(mtrait_link,names=mdic['matvars'])
     86 
     87     # create mesh

ipdb> mdic
{'vars': ('transparency',), 'func': 'pandas3js.views.jsmesh.transparent'}
ipdb> exit


In [7]:
size = 4
linesgeom = tjs.PlainGeometry(vertices=[[0, 0, 0],
                                    [size, 0, 0],
                                    [0, 0, 0],
                                    [0, size, 0],
                                    [0, 0, 0],
                                    [0, 0, size]],
                          colors = ['red', 'red', 'green', 'green', 'white', 'orange'])
lines = tjs.Line(geometry=linesgeom, 
             material=tjs.LineBasicMaterial(linewidth=5, vertexColors='VertexColors'), 
             type='LinePieces')
scene = tjs.Scene(children=[lines, tjs.DirectionalLight(color='#ccaabb', 
                            position=[0,10,0]),tjs.AmbientLight(color='#cccccc')])
c = tjs.PerspectiveCamera(position=[0, 10, 10])
renderer = tjs.Renderer(camera=c, background='black', background_opacity=1, 
                        scene = scene, controls=[tjs.OrbitControls(controlling=c)])
display(renderer)

In [9]:
import ipywidgets as widgets

collection = pjs.GeometricCollection()

scene = pjs.create_js_scene_view(collection,add_objects=True,add_labels=True)

camera, renderer = pjs.create_jsrenderer(scene,view=(1,-1,-1,1))

select = widgets.SelectionSlider(
    options=optsteps,
#    value=sortkeys[0],
    description='Configuration:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
#     readout_format='i',
     slider_color='black'
)
def format_label(s):
    s = ''.join([i for i in s if not i.isdigit()])
    return s.lower().capitalize()

def change_config(step,label=False):
    sdict = optimisation[step]['crystallographic']['geometry']
    df = pd.DataFrame(sdict)
    df.rename(columns={'x/a':'x','y/b':'y','z/c':'z'},
             inplace=True)
    #df['otype'] = 'Sphere'
    df.label = df.label.apply(format_label)
    df['radius'] = 0.1
    df['color'] = 'red'
    df.set_value(df['label']=='Fe','color','blue')
    df['label_color'] = df.color
    df['visible'] = not label
    df['label_visible'] = label
    collection.change_by_df(df,columns=['id','label','x','y','z','radius',
                                       'color', 'label_color','visible','label_visible'],
                            otype_default='pandas3js.Sphere')
toggle=widgets.Checkbox(
    value=False,
    description='View Label:',
    disabled=False
)
def handle_toggle(change):
    change_config(select.value, change.new)
toggle.observe(handle_toggle,names='value')

def handle_slider(change):
    change_config(change.new, toggle.value)
select.observe(handle_slider,names='value')
select.value = optsteps[-1]

widgets.VBox([widgets.HBox([select,toggle]),renderer])